In [30]:
import numpy as np
import pandas as pd
import random
import re

In [89]:
def arcs_to_vector(expression: re.Pattern, lines: list) -> pd.DataFrame:
    filtered = list(filter(expression.match, lines))
    vector = map(lambda x: x.split()[1:], filtered)
    colnames = ("tail", "head", "low", "cap", "cost")
    data = pd.DataFrame.from_records(list(vector), columns=colnames).astype(int)
    data["flow"] = 0
    return data

def special_nodes_to_arc(nodes: pd.DataFrame, value: int, origin=True):
    if origin:
        data = {
            "head": nodes["node"],
            "cap": nodes["flow"],
            "low": nodes["flow"],
        }
        data["tail"] = value
    else:
        data = {
            "tail": nodes["node"],
            "cap": -nodes["flow"],
            "low": -nodes["flow"],
        }
        data["head"] = value
    data["cost"] = 0
    data["flow"] = 0
    return pd.DataFrame(data, columns=("tail", "head", "low", "cap", "cost", "flow"))

def nodes_to_vector(expression: re.Pattern, lines: list) -> pd.DataFrame:
    filtered = list(filter(expression.match, lines))
    vector = map(lambda x: x.split()[1:], filtered)
    nodes = pd.DataFrame.from_records(list(vector), columns=("node", "flow")).astype(int)
    
    source_nodes = nodes["flow"] > 0
    sink_nodes = nodes["flow"] < 0
    
    return pd.concat([
        special_nodes_to_arc(nodes[source_nodes], 25001),
        special_nodes_to_arc(nodes[sink_nodes], 25002, origin=False),
    ])


def read_file(path_to_file: str) -> pd.DataFrame:
    r_arcs = re.compile("^a")
    r_nodes = re.compile("^n")
    with open(path_to_file) as f:
        content = f.readlines()
        arcs =  arcs_to_vector(r_arcs, content)
        nodes = nodes_to_vector(r_nodes, content)
    response = pd.concat([arcs, nodes], ignore_index=True)
    display(response)
    return response

In [90]:
net = read_file("../netg/stndrd1.net")

,tail,head,low,cap,cost,flow
0,1,102,0,100000,8550,0
1,1,171,0,100000,7415,0
2,1,101,0,100000,7178,0
3,1,165,0,100000,4494,0
4,1,151,0,100000,6289,0
...,...,...,...,...,...,...
1503,196,25002,96,96,0,0
1504,197,25002,755,755,0,0
1505,198,25002,1577,1577,0,0
1506,199,25002,131,131,0,0


In [78]:
display(net)

,index,tail,head,low,cap,cost,flow
0,0,1,102,0,100000,8550,0
1,1,1,171,0,100000,7415,0
2,2,1,101,0,100000,7178,0
3,3,1,165,0,100000,4494,0
4,4,1,151,0,100000,6289,0
...,...,...,...,...,...,...,...
1503,1503,196,25002,96,96,0,0
1504,1504,197,25002,755,755,0,0
1505,1505,198,25002,1577,1577,0,0
1506,1506,199,25002,131,131,0,0


In [61]:
def select_random(node: int, network: pd.DataFrame) -> pd.Series:
    mask_to = (network["tail"] == node) & (network["cap"] > network["flow"])
    selected = random.choice(network[mask_to].index)
    return network.loc[selected]

def run_pipe(start: int, end: int, network: pd.DataFrame, input_flow, output_flow):
    in_node = start
    flow_available = input_flow
    for i in range(10):
        go_to = select_random(in_node, network)
        print(go_to)
        print("\n")
        

In [62]:
sol = run_pipe(25001, 25002, net, 10000, -10000)

index       87
tail     25001
head        88
low        309
cap        309
cost         0
flow         0
Name: 1395, dtype: int64


index       30
tail     25001
head        31
low        947
cap        947
cost         0
flow         0
Name: 1338, dtype: int64


index       95
tail     25001
head        96
low       1914
cap       1914
cost         0
flow         0
Name: 1403, dtype: int64


index       55
tail     25001
head        56
low        301
cap        301
cost         0
flow         0
Name: 1363, dtype: int64


index       48
tail     25001
head        49
low         73
cap         73
cost         0
flow         0
Name: 1356, dtype: int64


index       25
tail     25001
head        26
low       1895
cap       1895
cost         0
flow         0
Name: 1333, dtype: int64


index       86
tail     25001
head        87
low        191
cap        191
cost         0
flow         0
Name: 1394, dtype: int64


index       78
tail     25001
head        79
low        939
cap        939
c

In [66]:
net

,index,tail,head,low,cap,cost,flow
0,0,1,102,0,100000,8550,0
1,1,1,171,0,100000,7415,0
2,2,1,101,0,100000,7178,0
3,3,1,165,0,100000,4494,0
4,4,1,151,0,100000,6289,0
...,...,...,...,...,...,...,...
1503,195,196,25002,96,96,0,0
1504,196,197,25002,755,755,0,0
1505,197,198,25002,1577,1577,0,0
1506,198,199,25002,131,131,0,0
